<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/CubicEOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap


In [ ]:
import sys
sys.path.insert(1, "/content/CHE2064")
import tools.che as che
import jax.numpy as jnp
import jax
from jax.config import config
config.update("jax_enable_x64", True)
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

In [2]:
R=8.314
props = che.Props('Water')

In [3]:
two_pi=2*jnp.pi
one_third = 1/3
def cubic_roots(a, b, c):
    # Returns only the real roots of cubic equations with real coefficients
    # x**3 + a x**2 + b x + c = 0

    Q = (a * a - 3 * b) / 9
    R = (2 * a * a * a - 9 * a * b + 27 * c) / 54
    det = (R * R - Q ** 3)

    if (det < 0):
      theta = jnp.arccos(R / pow(Q, 1.5))
      x=jnp.array((jnp.cos(theta/3), jnp.cos((theta+two_pi)/3), jnp.cos((theta-two_pi)/3)))
      x = -2 * jnp.sqrt(Q)*x - a/3
      return x
    else:
        A = -jnp.sign(R) * (abs(R) + jnp.sqrt(det)) ** one_third
        B = 0 if A == 0 else Q / A
        return jnp.array([(A + B) - a / 3, jnp.nan, jnp.nan])

In [4]:
def cubic_eos_P(V,T,props, eos='SRK'):
  Tr = T/props.Tc
  w = props.w
  alpha = {'SRK' : (1 + (0.48 + 1.574*w - 0.176*w**2)*(1-Tr**0.5))**2}
  sigma = {'SRK' : 1}
  epsilon = {'SRK' : 0}
  omega = {'SRK' : 0.08664}
  psi = {'SRK': 0.42748}
  a = psi[eos] * alpha[eos] * R**2 * props.Tc**2 / props.Pc
  b = omega[eos] * R * props.Tc / props.Pc
  return 8.314*T/(V-b) - a/((V+epsilon[eos]*b)*(V+sigma[eos]*b))

In [5]:
b = 0.08664 * R * props.Tc / props.Pc
V=jnp.logspace(jnp.log10(1.01*b),-1, 500)


/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning:

No GPU/TPU found, falling back to CPU.



In [6]:

fig=make_subplots(rows=1,cols=1)
for T in range(275,801,25):
  P=cubic_eos_P(V,T,props,eos='SRK')
  fig.add_scatter(x=V, y=P, mode='lines', name=f'{T}')

fig.update_layout(xaxis_title='$molar\ volume\ (m^3/mol)$',
                  yaxis_title='$Pressure\ (Pa)$')

fig.update_layout(xaxis_type='log', xaxis_range=(jnp.log10(b), -3), yaxis_range=(-50e6,200e6))
fig.show()


In [7]:
def cubic_Zv(P, T, props, eos='SRK'):
  Tr = T/props.Tc
  Pr = P/props.Pc
  w = props.w
  alpha = {'SRK' : (1 + (0.48 + 1.574*w - 0.176*w**2)*(1-Tr**0.5))**2}
  sigma = {'SRK' : 1}
  epsilon = {'SRK' : 0}
  omega = {'SRK' : 0.08664}
  psi = {'SRK': 0.42748}
  a = psi[eos] * alpha[eos] * R**2 * props.Tc**2 / props.Pc
  b = omega[eos] * R * props.Tc / props.Pc
  beta = b*P/(R*T)
  q = a/(b*R*T)

  return che.cubic_roots(beta*(epsilon[eos]+sigma[eos])-1-beta,
                         sigma[eos]*epsilon[eos]*beta**2 - (1+beta)*beta*(epsilon[eos]+sigma[eos])+q*beta,
                         -(1+beta)*sigma[eos]*epsilon[eos]*beta**2 - q*beta**2)

In [40]:
import numpy as np
u, w = np.polynomial.legendre.leggauss(10)

def quad(f, a, b):
  @jax.vmap
  def f_t(u):
    return f((u+1)*(b-a)/2 +a)
  return (b-a)/2*jnp.dot(w,f_t(u))

In [58]:
def cubic_Pvap(T, P):
  P_guess = 1e3


  def sum_sq_err_diff(P):
    f = lambda v: P - cubic_eos_P(v, T, props)
    v_roots = cubic_Zv(P, T, props, eos='SRK')*R*T/P
    print(quad(f,v_roots[1], v_roots[2]), quad(f,v_roots[0], v_roots[2]))

  return(sum_sq_err_diff(P))



In [71]:
cubic_Pvap(500,26.37e5)

2680.1168382613 2628.173389735656


In [70]:
props.Pvap(500)

DeviceArray(2636914.62607509, dtype=float64)